In [1]:
import pandas as pd
import glob
import os
import shutil 


In [2]:
def gridData(csvfiles):
    # Definition of the paths to folders and files:
    grid_path = 'C:/git/FB_ODC_2021/griglia_csv/GRIGLIA_MILANO.csv'  
    path_to_netcdf_folder = 'C:/git/FB_ODC_2021/netcdf_files'
    origin = 'C:/git/FB_ODC_2021/empty_yaml.yaml'
    
    # Upload the grid 
    grid = pd.read_csv(grid_path)
    
    #  All the csv are loaded as pandas dataframe, then are joined to the grid on quadkey value 
    gridded_csv = []
    
    for i in range(len(csvfiles)):
        # Temporary dataframe, from csv
        temp_df = pd.read_csv(csvfiles[i])
        # Not necessary columns are dropped
        temp_df = temp_df.drop(columns = ['country','lon', 'lat', 'n_baseline', 'n_difference', 'density_crisis', 'density_baseline', 'percent_change', 'clipped_z_score', 'ds'])
        # nan values are set to 0
        temp_df['n_crisis'] = temp_df['n_crisis'].replace( '\\N', 0)
        temp_df['n_crisis'] = temp_df['n_crisis'].astype(float)
        
        # merge of the csv with the grid on quadkey
        temp_gridded = grid.merge(temp_df, on = 'quadkey', how = 'outer')
        temp_gridded = temp_gridded.rename(columns = {'latitude_g':'latitude', 'longitude_g':'longitude'})
        temp_gridded['n_crisis'].fillna(0, inplace=True)
      
        # Datetimes are used to name the NETCDF files
        date_time = (((csvfiles[i].split('_'))[3]).split('.'))[0]  
        temp_gridded['date_time'] = datetime
        temp_gridded = temp_gridded.set_index(['quadkey'])
        # all gridded csv are stored in gridded_csv list of dataframe
        gridded_csv.append(temp_gridded)
        print('ok',i,'gridded')
        
        # Gridded dataframes are transformed in xarray
        temp_xarray = temp_gridded.to_xarray()
        print('ok',i,'to xarray')
        
        # xarray are transformed to netcdf files and saved in a specific folder
        netcdf_path = path_to_netcdf_folder+'/'+datetime+'.nc'
        temp_xarray.to_netcdf(netcdf_path)
        print('ok',i,'in netcdf')
    
        # Writing of the metadata on a yaml file
        datetime_string = datetime[0:10]+"T"+datetime[11]+datetime[12]+":"+datetime[13]+datetime[14]+":00.000Z"
        PID = list(datetime)
        PID.remove("-")
        PID.remove("-")
        PID.remove(" ")
        PID = "".join(PID)
        file1 = open(origin, "w")
        to_write = "$schema: https://schemas.opendatacube.org/dataset \n \nid: 00000000-0000-0000-0000-"+PID+"\n\nproduct:\n  name: FB_POI_MILANO\n  href: https://dataforgood.fb.com/ \n  format: NetCDF\n\ncrs: epsg:4326\n\ngeometry:\n  type: Polygon\n  coordinates: [[[ 8.995056152343800, 45.311597470877999], [8.995056152343800, 45.627484179430269], [9.549865722656120, 45.627484179430269], [9.549865722656120, 45.311597470877999], [ 8.995056152343800, 45.311597470877999]]]\n\ngrids:\n  default:\n    shape: [102,83] \n    transform: [1,0,0,0,1,0,0,0,1]\n\nlineage: {}\n\nmeasurements:\n  n_crisis:\n    layer: n_crisis\n    path: "+netcdf_path+"\n    nodata: -9999\n\nproperties:\n  odc:file_format: NetCDF\n  datetime: "+datetime_string
        file1.write(to_write)
        file1.close()
        target = "C:/git/FB_ODC_2021/cubeenv/dataset/"+PID+".yaml"
        # shutil library is used to save a copy of the file in the folder containing all metadata
        shutil.copy(origin, target)
        
        command = "datacube dataset add "+ target
        os.system(command)
        
    # In conclusion the names of the new peocessed csv are now written in loaded_csv.txt 
    to_write_on_txt = ",".join(csvfiles)+','
    with open("loaded_csv.txt", "a") as output:
        output.write(to_write_on_txt)
    
    
    
    return 'DONE!'

In [3]:
# Check if there are new csv files in /Coronavirus Disease Prevention/Population Map/milan to upload in OpenDataCube
fold_path = 'C:/git/FB_ODC_2021/milan'
csvfiles = []
for file in glob.glob(fold_path+"/"+"*.csv"):
    # We only consider not-empy files
    if os.stat(file).st_size != 0:
        csvfiles.append(file)

# 'loadedCSV.txt' contains all the names of the already loaded files, already_loaded is a list containing these files names
with open("C:/git/FB_ODC_2021/loaded_csv.txt", "r") as txt:
    already_loaded = (txt.read()).split(',')

# in order to check if there are new csv i use a loop that removes from csvfiles list the names of the already loaded csv
for name in already_loaded:
    if name in csvfiles:
        csvfiles.remove(name)  

if len(csvfiles) != 0:
    print(len(csvfiles), 'new CSVs have been found')
    print('Processing of CSVs is started...')
    ret = gridData(csvfiles)
    if ret == 'DONE!':
        print('CSVs have been processed and transformed in NETCDF format, metadata dataset have been created and correctly uploaded in ODC')
        
    else:
        print('Something whent wrong :(')
else:
    print('No new csv have been found, all csv are already uploaded in ODC')

No new csv have been found, all csv are already uploaded in ODC


# General information about ODC 

In [6]:
import datacube
#from odc.ui import DcViewer

In [7]:
dc = datacube.Datacube(app = "FB_ODC")

C:\Users\maria\anaconda3\envs\cubeenv\lib\site-packages\datacube\drivers\postgres\_connections.py:84: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


In [8]:
# Display all products contained in ODC database
list_of_products = dc.list_products()
products = dc.list_products()
display_columns = ["name",
                   "description",
                   "platform",
                   "instrument",
                   "crs",
                   "resolution"]

products[display_columns].sort_index()

,name,description,platform,instrument,crs,resolution
id,,,,,,
1,prova_1,FB for good data about population,None,None,EPSG:4326,None


In [9]:
# Select a specific product and show it's characteri
product = 'prova_1'
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata
measurement,,,,
n_crisis,n_crisis,float64,people,-9999
quadkey,quadkey,int64,adimentional,-9999


In [5]:
DcViewer(dc=dc,
         time='2020',
         width='800px',
         center=(45.469, 9.265),
         zoom=10)

# Loading data

In [16]:
#from odc.ui import with_ui_cbk

# The function dc.load allows to load all dataset from 'FB_POP_MILANO' that matches the given spatial and temporal extent
latitude_bounds = (45.311597470877999, 45.627484179430269)
longitude_bounds = (8.995056152343800, 9.549865722656120)
ds = dc.load(product='prova_1', 
             measurements = ['n_crisis'],
             time = '2020', 
             crs = 'EPSG:4326',
             resolution = (-0.001, 0.001), 
             output_crs='EPSG:4326',
             resampling='nearest')

C:\Users\maria\anaconda3\envs\cubeenv\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\maria\anaconda3\envs\cubeenv\lib\site-packages\datacube\storage\_rio.py:191: DeprecationWarning: Broken/missing geospatial data was found in file:
"NetCDF:"C:\git\FB_ODC_2021\netcdf_files\2020-04-050000.nc":n_crisis"
Will use approximate metadata for backwards compatibility reasons (#673).
This behaviour is deprecated. Future versions will raise an error.
  category=DeprecationWarning)
C:\Users\maria\anaconda3\envs\cubeenv\lib\site-packages\datacube\storage\_rio.py:191: DeprecationWarning: Broken/missing geospatial data was found in file:
"NetCDF:"C:\git\FB_ODC_2021\netcdf_files\2020-04-050800.nc":n_crisis"
Will use approximate metadata for backwards compatibility reasons (#673).
This behaviour is deprecated. Future versions will

In [17]:
print(ds)

<xarray.Dataset>
Dimensions:      (time: 4, latitude: 317, longitude: 555)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-05 ... 2020-04-06
  * latitude     (latitude) float64 45.63 45.63 45.63 ... 45.31 45.31 45.31
  * longitude    (longitude) float64 8.996 8.997 8.998 ... 9.548 9.549 9.55
    spatial_ref  int32 4326
Data variables:
    n_crisis     (time, latitude, longitude) float64 58.47 58.47 ... 65.82 65.82
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref


In [22]:
latitude_extents = (min(ds['latitude'].values),max(ds['latitude'].values))
longitude_extents = (min(ds['longitude'].values),max(ds['longitude'].values))
time_extents = (min(ds['time'].values),max(ds['time'].values))

In [23]:
print(latitude_extents)
print(longitude_extents)
print(time_extents)

(45.311499999999995, 45.6275)
(8.995500000000002, 9.549500000000002)
(numpy.datetime64('2020-04-05T00:00:00.000000000'), numpy.datetime64('2020-04-06T00:00:00.000000000'))


In [20]:
min(ds['n_crisis'].values)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()